In [ ]:
from collections import deque
import time

class Edge:
    def __init__(self, to, rev, capacity):
        self.to = to          # 边的终点
        self.rev = rev        # 反向边在邻接表中的索引
        self.capacity = capacity  # 边的剩余容量

class Dinic:
    def __init__(self, n):
        self.size = n
        self.graph = [[] for _ in range(n)]  # 邻接表存储图

    def add_edge(self, fr, to, cap):
        # 添加正向边和反向边
        forward = Edge(to, len(self.graph[to]), cap)
        backward = Edge(fr, len(self.graph[fr]), 0)
        self.graph[fr].append(forward)
        self.graph[to].append(backward)

    def bfs_level(self, s, t, level):
        # BFS 构建层次图
        q = deque()
        level[:] = [-1] * self.size
        level[s] = 0
        q.append(s)
        while q:
            v = q.popleft()
            for edge in self.graph[v]:
                if edge.capacity > 0 and level[edge.to] == -1:
                    level[edge.to] = level[v] + 1
                    q.append(edge.to)
                    if edge.to == t:
                        return

    def dfs_flow(self, v, t, upTo, iter_, level):
        # DFS 找阻塞流
        if v == t:
            return upTo
        for i in range(iter_[v], len(self.graph[v])):
            edge = self.graph[v][i]
            if edge.capacity > 0 and level[v] < level[edge.to]:
                d = self.dfs_flow(edge.to, t, min(upTo, edge.capacity), iter_, level)
                if d > 0:
                    edge.capacity -= d
                    self.graph[edge.to][edge.rev].capacity += d
                    return d
            iter_[v] += 1
        return 0

    def max_flow(self, s, t):
        # 计算最大流
        flow = 0
        level = [-1] * self.size
        while True:
            self.bfs_level(s, t, level)
            if level[t] == -1:
                return flow
            iter_ = [0] * self.size
            while True:
                f = self.dfs_flow(s, t, float('inf'), iter_, level)
                if f == 0:
                    break
                flow += f
            level = [-1] * self.size

def is_eliminated(teams, games, team_idx):
    """判断某支球队是否被淘汰，返回 (是否被淘汰, 理论最大胜场)"""
    n = len(teams)
    game_nodes = n * (n - 1) // 2  # 比赛节点数
    total_nodes = game_nodes + n + 2  # 总节点数：源点+比赛节点+球队节点+汇点
    s, t = 0, total_nodes - 1
    dinic = Dinic(total_nodes)

    # 1. 源点 → 比赛节点
    game_idx = 1
    for i in range(n):
        for j in range(i + 1, n):
            if games[i][j] > 0:
                dinic.add_edge(s, game_idx, games[i][j])  # 源点到比赛节点
                dinic.add_edge(game_idx, game_nodes + i + 1, 10**9)  # 比赛到球队i
                dinic.add_edge(game_idx, game_nodes + j + 1, 10**9)  # 比赛到球队j
                game_idx += 1

    # 2. 球队节点 → 汇点（计算理论最大胜场 & 容量限制）
    wx, rx = teams[team_idx]
    max_win_x = wx + rx  # 当前球队的理论最大胜场
    for i in range(n):
        wi, ri = teams[i]
        if i == team_idx:
            dinic.add_edge(game_nodes + i + 1, t, max_win_x)  # 自己的容量是理论最大胜场
        else:
            max_allow_y = min(wi + ri, max_win_x)  # 其他球队最多赢到max_win_x场
            dinic.add_edge(game_nodes + i + 1, t, max_allow_y - wi)  # 剩余可赢场数

    # 3. 计算最大流 & 总剩余比赛场数
    total_games = sum(games[i][j] for i in range(n) for j in range(i + 1, n) if games[i][j] > 0)
    max_flow_val = dinic.max_flow(s, t)
    is_elim = max_flow_val < total_games  # 是否被淘汰
    return is_elim, max_win_x

# 题目数据初始化
team_names = ["Atlanta", "Philly", "New York", "Montreal"]
teams = [
    (83, 8),   # Atlanta：已胜83，剩余8场 → 理论最大91
    (80, 3),   # Philly：已胜80，剩余3场 → 理论最大83
    (78, 6),   # New York：已胜78，剩余6场 → 理论最大84
    (77, 3)    # Montreal：已胜77，剩余3场 → 理论最大80
]
# games[i][j] 表示球队i和j之间剩余的比赛场数（i<j）
games = [
    [0, 1, 6, 1],  # Atlanta vs Philly(1), NY(6), Montreal(1)
    [0, 0, 0, 2],  # Philly vs NY(0), Montreal(2)
    [0, 0, 0, 0],  # NY vs Montreal(0)
    [0, 0, 0, 0]   # Montreal 无更多比赛
]

# 主程序：逐个判断球队 + 统计运行时间
start_time = time.time()
for i in range(4):
    is_elim, max_win = is_eliminated(teams, games, i)
    if is_elim:
        print(f"{team_names[i]}已经被淘汰了。最大的可能获胜场数{max_win}")
    else:
        print(f"{team_names[i]}没有被淘汰。最大的可能获胜场数{max_win}")
end_time = time.time()

# 输出总运行时间
print(f"\n总运行时间: {end_time - start_time:.6f} 秒")

In [ ]:
Dinic(G, s, t):
    // 初始化残留网络，与输入图 G 一致
    残留网络 residual_G = G  
    // 用于存储顶点层次的数组
    level = 数组，长度为顶点数，初始化为 -1  
    while True:
        // BFS 构造层次图，给顶点分配层次，同时判断是否还能增广
        成功 = BFS(residual_G, s, t, level)  
        if 成功为假:
            // 无法再增广，结束算法，返回最大流
            return 总流量  
        // 记录每个顶点已经处理到第几条边，避免重复检查
        ptr = 数组，长度为顶点数，初始化为 0  
        while True:
            // 在层次图中寻找阻塞流,start 辅助记录边的处理位置
            流量 = DFS(residual_G, s, t, 无穷大, level, ptr)  
            if 流量 == 0:
                // 无法找到新的阻塞流，退出当前层次图的处理
                break  
            // 累加本次阻塞流的流量到总流量
            总流量 += 流量  

In [ ]:
from collections import deque
import time

class Edge:
    def __init__(self, to, rev, capacity):
        self.to = to          # 边的终点
        self.rev = rev        # 反向边在邻接表中的索引
        self.capacity = capacity  # 边的剩余容量

class Dinic:
    def __init__(self, n):
        self.size = n
        self.graph = [[] for _ in range(n)]  # 邻接表存储图

    def add_edge(self, fr, to, cap):
        # 添加正向边和反向边
        forward = Edge(to, len(self.graph[to]), cap)
        backward = Edge(fr, len(self.graph[fr]), 0)
        self.graph[fr].append(forward)
        self.graph[to].append(backward)

    def bfs_level(self, s, t, level):
        # BFS 构建层次图
        q = deque()
        level[:] = [-1] * self.size
        level[s] = 0
        q.append(s)
        while q:
            v = q.popleft()
            for edge in self.graph[v]:
                if edge.capacity > 0 and level[edge.to] == -1:
                    level[edge.to] = level[v] + 1
                    q.append(edge.to)
                    if edge.to == t:
                        return

    def dfs_flow(self, v, t, upTo, iter_, level):
        # DFS 找阻塞流
        if v == t:
            return upTo
        for i in range(iter_[v], len(self.graph[v])):
            edge = self.graph[v][i]
            if edge.capacity > 0 and level[v] < level[edge.to]:
                d = self.dfs_flow(edge.to, t, min(upTo, edge.capacity), iter_, level)
                if d > 0:
                    edge.capacity -= d
                    self.graph[edge.to][edge.rev].capacity += d
                    return d
            iter_[v] += 1
        return 0

    def max_flow(self, s, t):
        # 计算最大流
        flow = 0
        level = [-1] * self.size
        while True:
            self.bfs_level(s, t, level)
            if level[t] == -1:
                return flow
            iter_ = [0] * self.size
            while True:
                f = self.dfs_flow(s, t, float('inf'), iter_, level)
                if f == 0:
                    break
                flow += f
            level = [-1] * self.size

def is_eliminated(teams, games, team_idx):
    """判断某支球队是否被淘汰，返回 (是否被淘汰, 理论最大胜场)"""
    n = len(teams)
    game_nodes = n * (n - 1) // 2  # 比赛节点数
    total_nodes = game_nodes + n + 2  # 总节点数：源点+比赛节点+球队节点+汇点
    s, t = 0, total_nodes - 1
    dinic = Dinic(total_nodes)

    # 1. 源点 → 比赛节点
    game_idx = 1
    for i in range(n):
        for j in range(i + 1, n):
            if games[i][j] > 0:
                dinic.add_edge(s, game_idx, games[i][j])  # 源点到比赛节点
                dinic.add_edge(game_idx, game_nodes + i + 1, 10**9)  # 比赛到球队i
                dinic.add_edge(game_idx, game_nodes + j + 1, 10**9)  # 比赛到球队j
                game_idx += 1

    # 2. 球队节点 → 汇点（计算理论最大胜场 & 容量限制）
    wx, rx = teams[team_idx]
    max_win_x = wx + rx  # 当前球队的理论最大胜场
    for i in range(n):
        wi, ri = teams[i]
        if i == team_idx:
            dinic.add_edge(game_nodes + i + 1, t, max_win_x)  # 自己的容量是理论最大胜场
        else:
            max_allow_y = min(wi + ri, max_win_x)  # 其他球队最多赢到max_win_x场
            dinic.add_edge(game_nodes + i + 1, t, max_allow_y - wi)  # 剩余可赢场数

    # 3. 计算最大流 & 总剩余比赛场数
    total_games = sum(games[i][j] for i in range(n) for j in range(i + 1, n) if games[i][j] > 0)
    max_flow_val = dinic.max_flow(s, t)
    is_elim = max_flow_val < total_games  # 是否被淘汰
    return is_elim, max_win_x

# 解析新数据集
team_names = ["New_York", "Baltimore", "Boston", "Toronto", "Detroit"]
# 球队数据：(已胜场数, 剩余场数)
teams = [
    (75, 28),   # New_York：已胜75，剩余28场 → 理论最大103
    (71, 28),   # Baltimore：已胜71，剩余28场 → 理论最大99
    (69, 27),   # Boston：已胜69，剩余27场 → 理论最大96
    (63, 27),   # Toronto：已胜63，剩余27场 → 理论最大90
    (49, 27)    # Detroit：已胜49，剩余27场 → 理论最大76
]

# 构建剩余比赛矩阵 games[i][j] 表示球队i和j之间剩余的比赛场数（i<j）
# 从数据中解析：
# New_York与其他队：0 3 8 7 3 → 提取后4个值（与Baltimore, Boston, Toronto, Detroit）
# Baltimore与其他队：3 0 2 7 7 → 提取后3个值（与Boston, Toronto, Detroit）
# Boston与其他队：8 2 0 0 3 → 提取后2个值（与Toronto, Detroit）
# Toronto与其他队：7 7 0 0 3 → 提取后1个值（与Detroit）
# Detroit与其他队：3 7 3 3 0 → 无（i<j）
games = [[0 for _ in range(5)] for _ in range(5)]
# 填充上三角矩阵
games[0][1] = 3  # New_York vs Baltimore
games[0][2] = 8  # New_York vs Boston
games[0][3] = 7  # New_York vs Toronto
games[0][4] = 3  # New_York vs Detroit
games[1][2] = 2  # Baltimore vs Boston
games[1][3] = 7  # Baltimore vs Toronto
games[1][4] = 7  # Baltimore vs Detroit
games[2][3] = 0  # Boston vs Toronto
games[2][4] = 3  # Boston vs Detroit
games[3][4] = 3  # Toronto vs Detroit

# 确保对称性（下三角等于上三角）
for i in range(5):
    for j in range(i+1, 5):
        games[j][i] = games[i][j]

# 主程序：逐个判断球队 + 统计运行时间
start_time = time.time()
for i in range(5):
    is_elim, max_win = is_eliminated(teams, games, i)
    if is_elim:
        print(f"{team_names[i]}已经被淘汰了。最大的可能获胜场数{max_win}")
    else:
        print(f"{team_names[i]}没有被淘汰。最大的可能获胜场数{max_win}")
end_time = time.time()

# 输出总运行时间
print(f"\n总运行时间: {end_time - start_time:.6f} 秒")

In [2]:
from collections import deque
import time

class Edge:
    def __init__(self, to, rev, capacity):
        self.to = to          # 边的终点
        self.rev = rev        # 反向边在邻接表中的索引
        self.capacity = capacity  # 边的剩余容量

class Dinic:
    def __init__(self, n):
        self.size = n
        self.graph = [[] for _ in range(n)]  # 邻接表存储图

    def add_edge(self, fr, to, cap):
        # 添加正向边和反向边
        forward = Edge(to, len(self.graph[to]), cap)
        backward = Edge(fr, len(self.graph[fr]), 0)
        self.graph[fr].append(forward)
        self.graph[to].append(backward)

    def bfs_level(self, s, t, level):
        # BFS 构建层次图
        q = deque()
        level[:] = [-1] * self.size
        level[s] = 0
        q.append(s)
        while q:
            v = q.popleft()
            for edge in self.graph[v]:
                if edge.capacity > 0 and level[edge.to] == -1:
                    level[edge.to] = level[v] + 1
                    q.append(edge.to)
                    if edge.to == t:
                        return

    def dfs_flow(self, v, t, upTo, iter_, level):
        # DFS 找阻塞流
        if v == t:
            return upTo
        for i in range(iter_[v], len(self.graph[v])):
            edge = self.graph[v][i]
            if edge.capacity > 0 and level[v] < level[edge.to]:
                d = self.dfs_flow(edge.to, t, min(upTo, edge.capacity), iter_, level)
                if d > 0:
                    edge.capacity -= d
                    self.graph[edge.to][edge.rev].capacity += d
                    return d
            iter_[v] += 1
        return 0

    def max_flow(self, s, t):
        # 计算最大流
        flow = 0
        level = [-1] * self.size
        while True:
            self.bfs_level(s, t, level)
            if level[t] == -1:
                return flow
            iter_ = [0] * self.size
            while True:
                f = self.dfs_flow(s, t, float('inf'), iter_, level)
                if f == 0:
                    break
                flow += f
            level = [-1] * self.size

def is_eliminated(teams, games, team_idx):
    """判断某支球队是否被淘汰，返回 (是否被淘汰, 理论最大胜场)"""
    n = len(teams)
    game_nodes = n * (n - 1) // 2  # 比赛节点数
    total_nodes = game_nodes + n + 2  # 总节点数：源点+比赛节点+球队节点+汇点
    s, t = 0, total_nodes - 1
    dinic = Dinic(total_nodes)

    # 1. 源点 → 比赛节点
    game_idx = 1
    for i in range(n):
        for j in range(i + 1, n):
            if games[i][j] > 0:
                dinic.add_edge(s, game_idx, games[i][j])  # 源点到比赛节点
                dinic.add_edge(game_idx, game_nodes + i + 1, 10**9)  # 比赛到球队i
                dinic.add_edge(game_idx, game_nodes + j + 1, 10**9)  # 比赛到球队j
                game_idx += 1

    # 2. 球队节点 → 汇点（计算理论最大胜场 & 容量限制）
    wx, rx = teams[team_idx]
    max_win_x = wx + rx  # 当前球队的理论最大胜场
    for i in range(n):
        wi, ri = teams[i]
        if i == team_idx:
            dinic.add_edge(game_nodes + i + 1, t, max_win_x)  # 自己的容量是理论最大胜场
        else:
            max_allow_y = min(wi + ri, max_win_x)  # 其他球队最多赢到max_win_x场
            dinic.add_edge(game_nodes + i + 1, t, max_allow_y - wi)  # 剩余可赢场数

    # 3. 计算最大流 & 总剩余比赛场数
    total_games = sum(games[i][j] for i in range(n) for j in range(i + 1, n) if games[i][j] > 0)
    max_flow_val = dinic.max_flow(s, t)
    is_elim = max_flow_val < total_games  # 是否被淘汰
    return is_elim, max_win_x

# 解析5支球队数据集
team_names = ["New_York", "Baltimore", "Boston", "Toronto", "Detroit"]
# 球队数据：(已胜场数, 剩余场数)
teams = [
    (75, 28),   # New_York：已胜75，剩余28场 → 理论最大103
    (71, 28),   # Baltimore：已胜71，剩余28场 → 理论最大99
    (69, 27),   # Boston：已胜69，剩余27场 → 理论最大96
    (63, 27),   # Toronto：已胜63，剩余27场 → 理论最大90
    (49, 27)    # Detroit：已胜49，剩余27场 → 理论最大76
]

# 构建剩余比赛矩阵 games[i][j] 表示球队i和j之间剩余的比赛场数（i<j）
games = [[0 for _ in range(5)] for _ in range(5)]
# 填充上三角矩阵
games[0][1] = 3  # New_York vs Baltimore
games[0][2] = 8  # New_York vs Boston
games[0][3] = 7  # New_York vs Toronto
games[0][4] = 3  # New_York vs Detroit
games[1][2] = 2  # Baltimore vs Boston
games[1][3] = 7  # Baltimore vs Toronto
games[1][4] = 7  # Baltimore vs Detroit
games[2][3] = 0  # Boston vs Toronto
games[2][4] = 3  # Boston vs Detroit
games[3][4] = 3  # Toronto vs Detroit

# 确保矩阵对称性（下三角等于上三角）
for i in range(5):
    for j in range(i+1, 5):
        games[j][i] = games[i][j]

def run_100_times():
    """运行100次并计算平均运行时间"""
    total_time = 0.0
    # 记录每次运行的结果，用于验证一致性
    all_results = []
    
    for run in range(100):
        start_time = time.time()
        
        # 每次运行都重新判断5支球队
        results = []
        for i in range(5):
            is_elim, max_win = is_eliminated(teams, games, i)
            results.append((is_elim, max_win))
        
        end_time = time.time()
        run_time = end_time - start_time
        total_time += run_time
        all_results.append(results)
        
        # 每10次运行输出一次进度
        if (run + 1) % 10 == 0:
            print(f"完成第{run + 1}次运行，本次耗时: {run_time:.6f}秒")
    
    # 验证所有运行结果是否一致
    first_result = all_results[0]
    consistent = all(result == first_result for result in all_results)
    
    # 计算平均运行时间
    avg_time = total_time / 100
    
    return avg_time, consistent, first_result

# 执行100次运行并获取结果
avg_time, consistent, results = run_100_times()

# 输出最终结果
print("\n--------------------- 运行结果 ---------------------")
for i in range(5):
    is_elim, max_win = results[i]
    if is_elim:
        print(f"{team_names[i]}已经被淘汰了。最大的可能获胜场数{max_win}")
    else:
        print(f"{team_names[i]}没有被淘汰。最大的可能获胜场数{max_win}")

print(f"\n100次运行的平均时间: {avg_time:.6f}秒")
print(f"所有运行结果是否一致: {'是' if consistent else '否'}")

完成第10次运行，本次耗时: 0.000360秒
完成第20次运行，本次耗时: 0.000261秒
完成第30次运行，本次耗时: 0.000262秒
完成第40次运行，本次耗时: 0.000311秒
完成第50次运行，本次耗时: 0.000331秒
完成第60次运行，本次耗时: 0.000258秒
完成第70次运行，本次耗时: 0.000392秒
完成第80次运行，本次耗时: 0.000258秒
完成第90次运行，本次耗时: 0.000258秒
完成第100次运行，本次耗时: 0.000340秒

--------------------- 运行结果 ---------------------
New_York没有被淘汰。最大的可能获胜场数103
Baltimore没有被淘汰。最大的可能获胜场数99
Boston没有被淘汰。最大的可能获胜场数96
Toronto没有被淘汰。最大的可能获胜场数90
Detroit已经被淘汰了。最大的可能获胜场数76

100次运行的平均时间: 0.000301秒
所有运行结果是否一致: 是


In [3]:
from collections import deque
import time

class Edge:
    def __init__(self, to, rev, capacity):
        self.to = to          # 边的终点
        self.rev = rev        # 反向边在邻接表中的索引
        self.capacity = capacity  # 边的剩余容量

class Dinic:
    def __init__(self, n):
        self.size = n
        self.graph = [[] for _ in range(n)]  # 邻接表存储图

    def add_edge(self, fr, to, cap):
        # 添加正向边和反向边
        forward = Edge(to, len(self.graph[to]), cap)
        backward = Edge(fr, len(self.graph[fr]), 0)
        self.graph[fr].append(forward)
        self.graph[to].append(backward)

    def bfs_level(self, s, t, level):
        # BFS 构建层次图
        q = deque()
        level[:] = [-1] * self.size
        level[s] = 0
        q.append(s)
        while q:
            v = q.popleft()
            for edge in self.graph[v]:
                if edge.capacity > 0 and level[edge.to] == -1:
                    level[edge.to] = level[v] + 1
                    q.append(edge.to)
                    if edge.to == t:
                        return

    def dfs_flow(self, v, t, upTo, iter_, level):
        # DFS 找阻塞流
        if v == t:
            return upTo
        for i in range(iter_[v], len(self.graph[v])):
            edge = self.graph[v][i]
            if edge.capacity > 0 and level[v] < level[edge.to]:
                d = self.dfs_flow(edge.to, t, min(upTo, edge.capacity), iter_, level)
                if d > 0:
                    edge.capacity -= d
                    self.graph[edge.to][edge.rev].capacity += d
                    return d
            iter_[v] += 1
        return 0

    def max_flow(self, s, t):
        # 计算最大流
        flow = 0
        level = [-1] * self.size
        while True:
            self.bfs_level(s, t, level)
            if level[t] == -1:
                return flow
            iter_ = [0] * self.size
            while True:
                f = self.dfs_flow(s, t, float('inf'), iter_, level)
                if f == 0:
                    break
                flow += f
            level = [-1] * self.size

def is_eliminated(teams, games, team_idx):
    """判断某支球队是否被淘汰，返回 (是否被淘汰, 理论最大胜场)"""
    n = len(teams)
    game_nodes = n * (n - 1) // 2  # 比赛节点数
    total_nodes = game_nodes + n + 2  # 总节点数：源点+比赛节点+球队节点+汇点
    s, t = 0, total_nodes - 1
    dinic = Dinic(total_nodes)

    # 1. 源点 → 比赛节点
    game_idx = 1
    for i in range(n):
        for j in range(i + 1, n):
            if games[i][j] > 0:
                dinic.add_edge(s, game_idx, games[i][j])  # 源点到比赛节点
                dinic.add_edge(game_idx, game_nodes + i + 1, 10**9)  # 比赛到球队i
                dinic.add_edge(game_idx, game_nodes + j + 1, 10**9)  # 比赛到球队j
                game_idx += 1

    # 2. 球队节点 → 汇点（计算理论最大胜场 & 容量限制）
    wx, rx = teams[team_idx]
    max_win_x = wx + rx  # 当前球队的理论最大胜场
    for i in range(n):
        wi, ri = teams[i]
        if i == team_idx:
            dinic.add_edge(game_nodes + i + 1, t, max_win_x)  # 自己的容量是理论最大胜场
        else:
            max_allow_y = min(wi + ri, max_win_x)  # 其他球队最多赢到max_win_x场
            dinic.add_edge(game_nodes + i + 1, t, max_allow_y - wi)  # 剩余可赢场数

    # 3. 计算最大流 & 总剩余比赛场数
    total_games = sum(games[i][j] for i in range(n) for j in range(i + 1, n) if games[i][j] > 0)
    max_flow_val = dinic.max_flow(s, t)
    is_elim = max_flow_val < total_games  # 是否被淘汰
    return is_elim, max_win_x

# 解析12支球队数据集
team_names = ["Poland", "Russia", "Brazil", "Iran", "Italy", "Cuba", "Argentina", "USA", "Japan", "Serbia", "Egypt", "China"]
# 球队数据：(已胜场数, 剩余场数)
teams = [
    (6, 4),    # Poland：已胜6，剩余4场 → 理论最大10
    (5, 5),    # Russia：已胜5，剩余5场 → 理论最大10
    (5, 5),    # Brazil：已胜5，剩余5场 → 理论最大10
    (5, 4),    # Iran：已胜5，剩余4场 → 理论最大9
    (4, 5),    # Italy：已胜4，剩余5场 → 理论最大9
    (4, 5),    # Cuba：已胜4，剩余5场 → 理论最大9
    (3, 5),    # Argentina：已胜3，剩余5场 → 理论最大8
    (3, 4),    # USA：已胜3，剩余4场 → 理论最大7
    (2, 4),    # Japan：已胜2，剩余4场 → 理论最大6
    (1, 5),    # Serbia：已胜1，剩余5场 → 理论最大6
    (1, 4),    # Egypt：已胜1，剩余4场 → 理论最大5
    (0, 4)     # China：已胜0，剩余4场 → 理论最大4
]

# 构建剩余比赛矩阵 games[i][j] 表示球队i和j之间剩余的比赛场数（i<j）
# 解析每支球队与其他球队的剩余比赛场数
# 注意：数据集每行最后14个数字中，前12个是与其他12支球队的比赛场数（包括自己，需忽略第一个0）
games = [[0 for _ in range(12)] for _ in range(12)]

# Poland的剩余比赛：0 1 1 0 1 0 0 0 0 0 1 0 → 提取后11个（与Russia, Brazil, Iran, Italy, Cuba, Argentina, USA, Japan, Serbia, Egypt, China）
games[0][1] = 1; games[0][2] = 1; games[0][3] = 0; games[0][4] = 1; games[0][5] = 0
games[0][6] = 0; games[0][7] = 0; games[0][8] = 0; games[0][9] = 0; games[0][10] = 1; games[0][11] = 0

# Russia的剩余比赛：1 0 0 1 0 1 1 0 1 0 0 0 → 提取后11个（与Brazil, Iran, Italy, Cuba, Argentina, USA, Japan, Serbia, Egypt, China）
games[1][2] = 0; games[1][3] = 1; games[1][4] = 0; games[1][5] = 1; games[1][6] = 1
games[1][7] = 0; games[1][8] = 1; games[1][9] = 0; games[1][10] = 0; games[1][11] = 0

# Brazil的剩余比赛：1 0 0 1 0 1 0 0 1 1 0 0 → 提取后11个（与Iran, Italy, Cuba, Argentina, USA, Japan, Serbia, Egypt, China）
games[2][3] = 1; games[2][4] = 0; games[2][5] = 1; games[2][6] = 0; games[2][7] = 0
games[2][8] = 1; games[2][9] = 1; games[2][10] = 0; games[2][11] = 0

# Iran的剩余比赛：0 1 1 0 1 0 0 0 0 0 0 1 → 提取后11个（与Italy, Cuba, Argentina, USA, Japan, Serbia, Egypt, China）
games[3][4] = 1; games[3][5] = 1; games[3][6] = 0; games[3][7] = 0; games[3][8] = 0
games[3][9] = 0; games[3][10] = 0; games[3][11] = 1

# Italy的剩余比赛：1 0 0 1 0 0 1 0 1 1 0 0 → 提取后11个（与Cuba, Argentina, USA, Japan, Serbia, Egypt, China）
games[4][5] = 0; games[4][6] = 1; games[4][7] = 0; games[4][8] = 1; games[4][9] = 1
games[4][10] = 0; games[4][11] = 0

# Cuba的剩余比赛：0 1 1 0 0 0 0 1 0 0 1 1 → 提取后11个（与Argentina, USA, Japan, Serbia, Egypt, China）
games[5][6] = 0; games[5][7] = 1; games[5][8] = 0; games[5][9] = 0; games[5][10] = 1; games[5][11] = 1

# Argentina的剩余比赛：0 1 0 0 1 0 0 1 0 0 1 1 → 提取后11个（与USA, Japan, Serbia, Egypt, China）
games[6][7] = 1; games[6][8] = 0; games[6][9] = 0; games[6][10] = 1; games[6][11] = 1

# USA的剩余比赛：0 0 0 0 0 1 1 0 1 1 0 0 → 提取后11个（与Japan, Serbia, Egypt, China）
games[7][8] = 1; games[7][9] = 1; games[7][10] = 0; games[7][11] = 0

# Japan的剩余比赛：0 1 1 0 1 0 0 1 0 0 0 0 → 提取后11个（与Serbia, Egypt, China）
games[8][9] = 0; games[8][10] = 0; games[8][11] = 0

# Serbia的剩余比赛：0 0 1 0 1 0 0 1 0 0 1 1 → 提取后11个（与Egypt, China）
games[9][10] = 1; games[9][11] = 1

# Egypt的剩余比赛：1 0 0 0 0 1 1 0 0 1 0 0 → 提取后11个（与China）
games[10][11] = 0

# 确保矩阵对称性（下三角等于上三角）
for i in range(12):
    for j in range(i+1, 12):
        games[j][i] = games[i][j]

# 主程序：逐个判断球队 + 统计运行时间
start_time = time.time()
for i in range(12):
    is_elim, max_win = is_eliminated(teams, games, i)
    if is_elim:
        print(f"{team_names[i]}已经被淘汰了。最大的可能获胜场数{max_win}")
    else:
        print(f"{team_names[i]}没有被淘汰。最大的可能获胜场数{max_win}")
end_time = time.time()

# 输出总运行时间
print(f"\n总运行时间: {end_time - start_time:.6f} 秒")

Poland没有被淘汰。最大的可能获胜场数10
Russia没有被淘汰。最大的可能获胜场数10
Brazil没有被淘汰。最大的可能获胜场数10
Iran没有被淘汰。最大的可能获胜场数9
Italy没有被淘汰。最大的可能获胜场数9
Cuba没有被淘汰。最大的可能获胜场数9
Argentina没有被淘汰。最大的可能获胜场数8
USA没有被淘汰。最大的可能获胜场数7
Japan已经被淘汰了。最大的可能获胜场数6
Serbia已经被淘汰了。最大的可能获胜场数6
Egypt已经被淘汰了。最大的可能获胜场数5
China已经被淘汰了。最大的可能获胜场数4

总运行时间: 0.002376 秒


In [5]:
from collections import deque
import time

class Edge:
    def __init__(self, to, rev, capacity):
        self.to = to          # 边的终点
        self.rev = rev        # 反向边在邻接表中的索引
        self.capacity = capacity  # 边的剩余容量

class Dinic:
    def __init__(self, n):
        self.size = n
        self.graph = [[] for _ in range(n)]  # 邻接表存储图

    def add_edge(self, fr, to, cap):
        # 添加正向边和反向边
        forward = Edge(to, len(self.graph[to]), cap)
        backward = Edge(fr, len(self.graph[fr]), 0)
        self.graph[fr].append(forward)
        self.graph[to].append(backward)

    def bfs_level(self, s, t, level):
        # BFS 构建层次图
        q = deque()
        level[:] = [-1] * self.size
        level[s] = 0
        q.append(s)
        while q:
            v = q.popleft()
            for edge in self.graph[v]:
                if edge.capacity > 0 and level[edge.to] == -1:
                    level[edge.to] = level[v] + 1
                    q.append(edge.to)
                    if edge.to == t:
                        return

    def dfs_flow(self, v, t, upTo, iter_, level):
        # DFS 找阻塞流
        if v == t:
            return upTo
        for i in range(iter_[v], len(self.graph[v])):
            edge = self.graph[v][i]
            if edge.capacity > 0 and level[v] < level[edge.to]:
                d = self.dfs_flow(edge.to, t, min(upTo, edge.capacity), iter_, level)
                if d > 0:
                    edge.capacity -= d
                    self.graph[edge.to][edge.rev].capacity += d
                    return d
            iter_[v] += 1
        return 0

    def max_flow(self, s, t):
        # 计算最大流
        flow = 0
        level = [-1] * self.size
        while True:
            self.bfs_level(s, t, level)
            if level[t] == -1:
                return flow
            iter_ = [0] * self.size
            while True:
                f = self.dfs_flow(s, t, float('inf'), iter_, level)
                if f == 0:
                    break
                flow += f
            level = [-1] * self.size

def is_eliminated(teams, games, team_idx):
    """判断某支球队是否被淘汰，返回 (是否被淘汰, 理论最大胜场)"""
    n = len(teams)
    game_nodes = n * (n - 1) // 2  # 比赛节点数
    total_nodes = game_nodes + n + 2  # 总节点数：源点+比赛节点+球队节点+汇点
    s, t = 0, total_nodes - 1
    dinic = Dinic(total_nodes)

    # 1. 源点 → 比赛节点
    game_idx = 1
    for i in range(n):
        for j in range(i + 1, n):
            if games[i][j] > 0:
                dinic.add_edge(s, game_idx, games[i][j])  # 源点到比赛节点
                dinic.add_edge(game_idx, game_nodes + i + 1, 10**9)  # 比赛到球队i
                dinic.add_edge(game_idx, game_nodes + j + 1, 10**9)  # 比赛到球队j
                game_idx += 1

    # 2. 球队节点 → 汇点（计算理论最大胜场 & 容量限制）
    wx, rx = teams[team_idx]
    max_win_x = wx + rx  # 当前球队的理论最大胜场
    for i in range(n):
        wi, ri = teams[i]
        if i == team_idx:
            dinic.add_edge(game_nodes + i + 1, t, max_win_x)  # 自己的容量是理论最大胜场
        else:
            max_allow_y = min(wi + ri, max_win_x)  # 其他球队最多赢到max_win_x场
            dinic.add_edge(game_nodes + i + 1, t, max_allow_y - wi)  # 剩余可赢场数

    # 3. 计算最大流 & 总剩余比赛场数
    total_games = sum(games[i][j] for i in range(n) for j in range(i + 1, n) if games[i][j] > 0)
    max_flow_val = dinic.max_flow(s, t)
    is_elim = max_flow_val < total_games  # 是否被淘汰
    return is_elim, max_win_x

# 解析12支球队数据集
team_names = ["Poland", "Russia", "Brazil", "Iran", "Italy", "Cuba", "Argentina", "USA", "Japan", "Serbia", "Egypt", "China"]
# 球队数据：(已胜场数, 剩余场数)
teams = [
    (6, 4),    # Poland：已胜6，剩余4场 → 理论最大10
    (5, 5),    # Russia：已胜5，剩余5场 → 理论最大10
    (5, 5),    # Brazil：已胜5，剩余5场 → 理论最大10
    (5, 4),    # Iran：已胜5，剩余4场 → 理论最大9
    (4, 5),    # Italy：已胜4，剩余5场 → 理论最大9
    (4, 5),    # Cuba：已胜4，剩余5场 → 理论最大9
    (3, 5),    # Argentina：已胜3，剩余5场 → 理论最大8
    (3, 4),    # USA：已胜3，剩余4场 → 理论最大7
    (2, 4),    # Japan：已胜2，剩余4场 → 理论最大6
    (1, 5),    # Serbia：已胜1，剩余5场 → 理论最大6
    (1, 4),    # Egypt：已胜1，剩余4场 → 理论最大5
    (0, 4)     # China：已胜0，剩余4场 → 理论最大4
]

# 构建剩余比赛矩阵 games[i][j] 表示球队i和j之间剩余的比赛场数（i<j）
games = [[0 for _ in range(12)] for _ in range(12)]

# 解析每支球队与其他球队的剩余比赛场数（数据集每行最后14个数字中，前12个是与其他12支球队的比赛场数，跳过第一个0）
# Poland: 0 1 1 0 1 0 0 0 0 0 1 0 → 索引1-11
games[0][1] = 1; games[0][2] = 1; games[0][3] = 0; games[0][4] = 1; games[0][5] = 0
games[0][6] = 0; games[0][7] = 0; games[0][8] = 0; games[0][9] = 0; games[0][10] = 1; games[0][11] = 0

# Russia: 1 0 0 1 0 1 1 0 1 0 0 0 → 索引2-11
games[1][2] = 0; games[1][3] = 1; games[1][4] = 0; games[1][5] = 1; games[1][6] = 1
games[1][7] = 0; games[1][8] = 1; games[1][9] = 0; games[1][10] = 0; games[1][11] = 0

# Brazil: 1 0 0 1 0 1 0 0 1 1 0 0 → 索引3-11
games[2][3] = 1; games[2][4] = 0; games[2][5] = 1; games[2][6] = 0; games[2][7] = 0
games[2][8] = 1; games[2][9] = 1; games[2][10] = 0; games[2][11] = 0

# Iran: 0 1 1 0 1 0 0 0 0 0 0 1 → 索引4-11
games[3][4] = 1; games[3][5] = 1; games[3][6] = 0; games[3][7] = 0; games[3][8] = 0
games[3][9] = 0; games[3][10] = 0; games[3][11] = 1

# Italy: 1 0 0 1 0 0 1 0 1 1 0 0 → 索引5-11
games[4][5] = 0; games[4][6] = 1; games[4][7] = 0; games[4][8] = 1; games[4][9] = 1
games[4][10] = 0; games[4][11] = 0

# Cuba: 0 1 1 0 0 0 0 1 0 0 1 1 → 索引6-11
games[5][6] = 0; games[5][7] = 1; games[5][8] = 0; games[5][9] = 0; games[5][10] = 1; games[5][11] = 1

# Argentina: 0 1 0 0 1 0 0 1 0 0 1 1 → 索引7-11
games[6][7] = 1; games[6][8] = 0; games[6][9] = 0; games[6][10] = 1; games[6][11] = 1

# USA: 0 0 0 0 0 1 1 0 1 1 0 0 → 索引8-11
games[7][8] = 1; games[7][9] = 1; games[7][10] = 0; games[7][11] = 0

# Japan: 0 1 1 0 1 0 0 1 0 0 0 0 → 索引9-11
games[8][9] = 0; games[8][10] = 0; games[8][11] = 0

# Serbia: 0 0 1 0 1 0 0 1 0 0 1 1 → 索引10-11
games[9][10] = 1; games[9][11] = 1

# Egypt: 1 0 0 0 0 1 1 0 0 1 0 0 → 索引11-11
games[10][11] = 0

# 确保矩阵对称性（下三角等于上三角）
for i in range(12):
    for j in range(i+1, 12):
        games[j][i] = games[i][j]

def run_100_times():
    """运行100次并计算平均运行时间"""
    total_time = 0.0
    # 记录每次运行的结果，用于验证一致性
    all_results = []
    
    for run in range(100):
        start_time = time.time()
        
        # 每次运行都重新判断12支球队
        results = []
        for i in range(12):
            is_elim, max_win = is_eliminated(teams, games, i)
            results.append((is_elim, max_win))
        
        end_time = time.time()
        run_time = end_time - start_time
        total_time += run_time
        all_results.append(results)
        
        # 每10次运行输出一次进度
        if (run + 1) % 10 == 0:
            print(f"完成第{run + 1}次运行，本次耗时: {run_time:.6f}秒")
    
    # 验证所有运行结果是否一致
    first_result = all_results[0]
    consistent = all(result == first_result for result in all_results)
    
    # 计算平均运行时间
    avg_time = total_time / 100
    
    return avg_time, consistent, first_result

# 执行100次运行并获取结果
avg_time, consistent, results = run_100_times()

# 输出最终结果
print("\n--------------------- 运行结果 ---------------------")
for i in range(12):
    is_elim, max_win = results[i]
    if is_elim:
        print(f"{team_names[i]}已经被淘汰了。最大的可能获胜场数{max_win}")
    else:
        print(f"{team_names[i]}没有被淘汰。最大的可能获胜场数{max_win}")

print(f"\n100次运行的平均时间: {avg_time:.6f}秒")
print(f"所有运行结果是否一致: {'是' if consistent else '否'}")

完成第10次运行，本次耗时: 0.001799秒
完成第20次运行，本次耗时: 0.001761秒
完成第30次运行，本次耗时: 0.001971秒
完成第40次运行，本次耗时: 0.001774秒
完成第50次运行，本次耗时: 0.001810秒
完成第60次运行，本次耗时: 0.001815秒
完成第70次运行，本次耗时: 0.003189秒
完成第80次运行，本次耗时: 0.001852秒
完成第90次运行，本次耗时: 0.001780秒
完成第100次运行，本次耗时: 0.001792秒

--------------------- 运行结果 ---------------------
Poland没有被淘汰。最大的可能获胜场数10
Russia没有被淘汰。最大的可能获胜场数10
Brazil没有被淘汰。最大的可能获胜场数10
Iran没有被淘汰。最大的可能获胜场数9
Italy没有被淘汰。最大的可能获胜场数9
Cuba没有被淘汰。最大的可能获胜场数9
Argentina没有被淘汰。最大的可能获胜场数8
USA没有被淘汰。最大的可能获胜场数7
Japan已经被淘汰了。最大的可能获胜场数6
Serbia已经被淘汰了。最大的可能获胜场数6
Egypt已经被淘汰了。最大的可能获胜场数5
China已经被淘汰了。最大的可能获胜场数4

100次运行的平均时间: 0.001978秒
所有运行结果是否一致: 是
